<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part of the free course [EEwPython](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html). 99% of this material has been adapted from [Google Earth Engine Guides](https://developers.google.com/earth-engine/).

<!--NAVIGATION-->
 < [ImageCollection](3_eeImageCollection.ipynb) | [Contents](index.ipynb) |  [Reducer](5_Reducer.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/4_features.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<center>
<h1>Google Earth Engine with Python </h1>
<h2> ee.Geometry, ee.Feature and ee.FeatureCollection </h2>
</center>
<h2> Topics:</h2>

1. Geometry Overview
2. Geodesic vs. Planar Geometries
3. Geometry information and metadata
4. Geometric Operations
5.Feature Overview
6.FeatureCollection Overview
7. FeatureCollection Information and Metadata
8. Filtering a FeatureCollection
9. Mapping over a FeatureCollection
10. Reducing a FeatureCollection



## Connecting GEE with Google Services

- **Authenticate to Earth Engine**

In [0]:
!pip install earthengine-api #earth-engine Python API

In [0]:
!earthengine authenticate 

- **Authenticate to Google Drive (OPTIONAL)**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

- **Authenticate to Google Cloud (OPTIONAL)**

In [0]:
from google.colab import auth
auth.authenticate_user()

## Testing the software setup


In [0]:
# Earth Engine Python API
import ee
ee.Initialize()

In [4]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

Folium version: 0.8.3


In [0]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

# 1.  Geometry overview

Earth Engine handles vector data with the `Geometry` type. The [GeoJSON spec](https://geojson.org/geojson-spec.html) describes in detail the type of geometries supported by Earth Engine, including `Point` (a list of coordinates in some projection), `LineString` (a list of points), `LinearRing` (a closed `LineString`), and `Polygon` (a list of `LinearRings` where the first is a shell and subsequent rings are holes). Earth Engine also supports `MultiPoint`, `MultiLineString`, and `MultiPolygon`. The GeoJSON GeometryCollection is also supported, although it has the name `MultiGeometry` within Earth Engine.


#### Creating Geometry objects
 To create a Geometry programmatically, provide the constructor with the proper list(s) of coordinates. For example:

In [0]:
point = ee.Geometry.Point([1.5, 1.5])
lineString = ee.Geometry.LineString([[-35, -10], [35, -10], [35, 10], [-35, 10]])
linearRing = ee.Geometry.LinearRing([[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]])
rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20])
polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

In the previous examples, note that the distinction between a `LineString` and a `LinearRing` is that the `LinearRing` is “closed” by having the same coordinate at both the start and end of the list.

An individual `Geometry` may consist of multiple geometries. To break a multi-part `Geometry` into its constituent geometries, use `geometry.geometries()`. For example:

In [0]:
# Create a multi-part feature.
multiPoint = ee.Geometry.MultiPoint([[-121.68, 39.91], [-97.38, 40.34]])

# Get the individual geometries as a list.
geometries = multiPoint.geometries()

# Get each individual geometry from the list and print it.
pt1 = geometries.get(0)
pt2 = geometries.get(1)
print('Point 1', pt1)
print('Point 2', pt2)

# 2. Geodesic vs. Planar Geometries

A geometry created in Earth Engine is either geodesic (i.e. edges are the shortest path on the surface of a sphere) or planar (i.e. edges are the shortest path in a 2-D Cartesian plane). No one planar coordinate system is suitable for global collections of features, so Earth Engine's geometry constructors build geodesic geometries by default. To make a planar geometry, constructors have a geodesic parameter that can be set to false:



In [0]:
planarPolygon = ee.Geometry(polygon, None, False)

Figure 1 shows the difference between the default geodesic polygon and the result of converting the polygon to a planar representation.


<center>
<img src="https://developers.google.com/earth-engine/images/Geometry_geodesic_vs_planar_annotated.png">  
</center>

# 3.  Geometry information and metadata

To view information about a geometry, print it. To access the information programmatically, Earth Engine provides several methods. For example, to get information about the polygon created previously, use:

In [0]:
print('Polygon printout: ', polygon.getInfo())

# Print polygon area in square kilometers.
print('Polygon area: ', polygon.area().divide(1000 * 1000).getInfo())

# Print polygon perimeter length in kilometers.
print('Polygon perimeter: ', polygon.perimeter().divide(1000).getInfo())

# Print the geometry as a GeoJSON string.
print('Polygon GeoJSON: ', polygon.toGeoJSONString())

# Print the GeoJSON 'type'
print('Geometry type: ', polygon.type().getInfo())

# Print the coordinates as lists
print('Polygon coordinates: ', polygon.coordinates().getInfo())

# Print whether the geometry is geodesic.
print('Geodesic? ', polygon.geodesic().getInfo())      

Polygon printout:  {'type': 'Polygon', 'coordinates': [[[-5.0, 40.0], [65.0, 40.0], [65.0, 60.0], [-5.0, 60.0], [-5.0, 60.0], [-5.0, 40.0]]]}
Polygon area:  9918985.748240016
Polygon perimeter:  13979.88727044261
Polygon GeoJSON:  {"type": "Polygon", "coordinates": [[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]], "evenOdd": true}
Geometry type:  Polygon
Polygon coordinates:  [[[-5.0, 40.0], [65.0, 40.0], [65.0, 60.0], [-5.0, 60.0], [-5.0, 60.0], [-5.0, 40.0]]]
Geodesic?  True


Observe that the perimeter (or length) of a geometry is returned in meters and the area is returned in square meters unless a projection is specified. By default, the computation is performed on the WGS84 spheroid and the result is computed in meters or square meters.

# 4. Geometric Operations

Earth Engine supports a wide variety of operations on `Geometry` objects. These include operations on individual geometries such as computing a buffer, centroid, bounding box, perimeter, convex hull, etc. For example:



In [20]:
# Create a geodesic polygon.
polygon = ee.Geometry.Polygon(coords = [[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

# Compute a buffer of the polygon.
buffer = polygon.buffer(1000000).getInfo()

# Compute the centroid of the polygon.
centroid = polygon.centroid().getInfo()


#Go Folium
center = centroid['coordinates']

Mapdisplay(center,{'center':centroid,'buffer':buffer},zoom_start=3)

Observe from the previous example that the buffer distance is specified in meters.

Supported geometric operations also include relational computations between geometries such as intersection, union, difference, distance, contains, etc. To test some of these relations, geometries use the “even-odd” rule by default. By the even-odd rule, a point is inside the polygon if a line from that point to some point known to be outside the polygon crosses an odd number of other edges. The inside of a polygon is everything inside the shell and not inside a hole. As a simple example, a point within a circular polygon must cross exactly one edge to escape the polygon. Geometries can optionally use the "left-inside" rule, if necessary. Imagine walking the points of a ring in the order given; the inside will be on the left.

To demonstrate the difference between geometries created with the "left-inside" rule (`evenOdd: false`) and those created with the "even-odd" rule, the following example compares a point to two different polygons:

In [0]:
# Create a right-inside polygon.
holePoly = ee.Geometry.Polygon(coords = [[[-35, -10], [-35, 10], [35, 10], [35, -10], [-35, -10]]],
                               proj= 'EPSG:4326',
                               geodesic = True,
                               maxError= 1.,
                               evenOdd = False)

# Create an even-odd version of the polygon.
evenOddPoly = ee.Geometry(geo_json =  holePoly.getInfo(),
                          opt_proj = 'EPSG:4326',
                          opt_evenOdd = True)

# Create a point to test the insideness of the polygon.
pt = ee.Geometry.Point([1.5, 1.5])

# Check insideness with a contains operator.
print(holePoly.contains(pt).getInfo())       # False
print(evenOddPoly.contains(pt).getInfo())    # True

False
True


The previous example demonstrates how the order of coordinates provided to the Polygon constructor affects the result when a left-inside polygon is constructed. Specifically, the point is outside the left-inside polygon but inside the even-odd polygon.

The following example computes and visualizes derived geometries based on the relationship between two polygons:

In [27]:
# Create two circular geometries.
poly1 = ee.Geometry.Point([-50, 30]).buffer(1e6)
poly2 = ee.Geometry.Point([-40, 30]).buffer(1e6)

poly1_geojson = poly1.getInfo()
poly2_geojson = poly2.getInfo()

# Display polygon 1 in red and polygon 2 in blue.
Mapdisplay(center= (30,-45),dicc={'poly1':poly1_geojson,'poly2':poly2_geojson},zoom_start=3)

In [28]:
# Compute the intersection, display it in blue.
intersection = poly1.intersection(poly2, ee.ErrorMargin(1)).getInfo()
Mapdisplay(center= (30,-45),dicc={'intersection':intersection},zoom_start=3)

In [30]:
# Compute the union, display it in magenta.
union = poly1.union(poly2, ee.ErrorMargin(1)).getInfo()
Mapdisplay(center= (30,-45),dicc={'union':union},zoom_start=3)

In [31]:
# Compute the difference, display in yellow.
diff1 = poly1.difference(poly2, ee.ErrorMargin(1)).getInfo()
Mapdisplay(center= (30,-45),dicc={'diff':diff1},zoom_start=3)

In [32]:
# Compute symmetric difference, display in black.
symDiff = poly1.symmetricDifference(poly2, ee.ErrorMargin(1)).getInfo()
Mapdisplay(center= (30,-45),dicc={'symDiff':symDiff},zoom_start=3)

In these examples, note that that maxError parameter is set to one meter for the geometry operations. The maxError is the maximum allowable error, in meters, from transformations (such as projection or reprojection) that may alter the geometry. If one of the geometries is in a different projection from the other, Earth Engine will do the computation in a spherical coordinate system, with a projection precision given by maxError. You can also specify a specific projection in which to do the computation, if necessary.



# 5. Feature Overview

A `Feature` in Earth Engine is defined as a GeoJSON `Feature`. Specifically, a `Feature` is an object with a geometry property storing a `Geometry` object (or null) and a `properties` property storing a dictionary of other properties.

## Creating Feature objects
To create a `Feature`, provide the constructor with a `Geometry` and (optionally) a dictionary of other properties. For example:

In [0]:
# Create an ee.Geometry.
polygon = ee.Geometry.Polygon([[[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]])

# Create a Feature from the Geometry.
polyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart'})

As with a `Geometry`, a `Feature` may be printed or added to the map for inspection and visualization:

In [0]:
polyFeature.getInfo()

{'geometry': {'coordinates': [[[-35.0, -10.0],
    [35.0, -10.0],
    [35.0, 10.0],
    [-35.0, 10.0],
    [-35.0, -10.0]]],
  'type': 'Polygon'},
 'properties': {'bar': 'tart', 'foo': 42},
 'type': 'Feature'}

A `Feature` need not have a Geometry and may simply wrap a dictionary of properties. For example:



In [0]:
# Create a dictionary of properties, some of which may be computed values.
dict = {'foo': ee.Number(8).add(88), 'bar': 'nihao'}

# Create a null geometry feature with the dictionary of properties.
nowhereFeature = ee.Feature(None, dict)

In this example, note that the dictionary supplied to the `Feature` contains a computed value. Creating features in this manner is useful for exporting long-running computations with a `Dictionary` result (e.g. `image.reduceRegion()`). See the [FeatureCollections](https://developers.google.com/earth-engine/feature_collections) and [Importing](https://developers.google.com/earth-engine/importing) or [Exporting](https://developers.google.com/earth-engine/exporting) sections for details.

Each `Feature` has one primary `Geometry` stored in the `geometry` property. Additional geometries may be stored in other properties. `Geometry` methods such as intersection and buffer also exist on `Feature` as a convenience for getting the primary `Geometry`, applying the operation, and setting the result as the new primary `Geometry`. The result will retain all the other properties of the `Feature` on which the method is called. There are also methods for getting and setting the non-geometry properties of the `Feature`. For example:

In [33]:
from pprint import pprint
# Make a feature and set some properties.
feature = ee.Feature(ee.Geometry.Point([-122.22599, 37.17605]))\
            .set('genus', 'Sequoia').set('species', 'sempervirens')

# Get a property from the feature.
species = feature.get('species')
print(species.getInfo())

# Set a new property.
feature = feature.set('presence', 1)

# Overwrite the old properties with a new dictionary.
newDict = {'genus': 'Brachyramphus', 'species': 'marmoratus'}
feature = feature.set(newDict)

# Check the result.
pprint(feature.getInfo())

sempervirens
{'geometry': {'coordinates': [-122.22599, 37.17605], 'type': 'Point'},
 'properties': {'genus': 'Brachyramphus',
                'presence': 1,
                'species': 'marmoratus'},
 'type': 'Feature'}


In the previous example, note that properties can be set with either a key: value pair, or with a dictionary as a JavaScript literal. Also note that `feature.set()` overwrites existing properties.

# 6. FeatureCollection Overview

Groups of related features can be combined into a `FeatureCollection`, to enable additional operations on the entire set such as filtering, sorting and rendering. Besides just simple features (geometry + properties), feature collections can also contain other collections.

#### The FeatureCollection constructor

One way to create a `FeatureCollection` is to provide the constructor with a list of features. The features do not need to have the same geometry type or the same properties. For example:

In [0]:
# Make a list of Features.
features = [
  ee.Feature(ee.Geometry.Rectangle(30.01, 59.80, 30.59, 60.15), {'name': 'Voronoi'}),
  ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
  ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'})
]

# Create a FeatureCollection from the list and print it.
fromList = ee.FeatureCollection(features)
fromList.getInfo()

{'columns': {'name': 'String', 'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [[[30.01, 59.8],
      [30.59, 59.8],
      [30.59, 60.15],
      [30.01, 60.15],
      [30.01, 59.8]]],
    'type': 'Polygon'},
   'id': '0',
   'properties': {'name': 'Voronoi'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.96, 40.781], 'type': 'Point'},
   'id': '1',
   'properties': {'name': 'Thiessen'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [6.4806, 50.8012], 'type': 'Point'},
   'id': '2',
   'properties': {'name': 'Dirichlet'},
   'type': 'Feature'}],
 'type': 'FeatureCollection'}

Individual geometries can also be turned into a `FeatureCollection` of just one Feature:



In [0]:
# Create a FeatureCollection from a single geometry and print it.
fromGeom = ee.FeatureCollection(ee.Geometry.Point(16.37, 48.225))
fromGeom.getInfo()

{'columns': {'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [16.37, 48.225], 'type': 'Point'},
   'id': '0',
   'properties': {},
   'type': 'Feature'}],
 'type': 'FeatureCollection'}

## Table Datasets

Earth Engine hosts a variety of table datasets. To load a table dataset, provide the table ID to the `FeatureCollection` constructor. For example, to load TIGER roads data:

In [48]:
fc = ee.FeatureCollection('TIGER/2016/Roads')
road = fc.limit(50)
centroid = road.geometry().centroid().getInfo()['coordinates']
centroid.reverse()
Mapdisplay(centroid,{'road':road.getInfo()},zoom_start=12)

Note that as with image datasets, you can search for table datasets and import them into your script using the Code Editor search tool. There is a list of selected vector datasets hosted by Earth Engine on this page.

## Fusion Tables (deprecated)

Earth Engine can also use tables stored in Google Fusion Tables. To load a `FeatureCollection` from a Fusion Table, supply the constructor with the table ID appended to `ft:`. For example:

In [0]:
# Load a FeatureCollection from a Fusion Table.
fromFT = ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw')

# Print and display the FeatureCollection.
print(fromFT)

Note that the ID of the Fusion Table is everything after the ft: and before the closing quote. To find this ID from the Fusion Tables interface, go to File > About this table and copy the encrypted ID from the ‘Id’ section. You can also find the ID after the ‘docid=’ in the table’s URL. Learn more about using Fusion Tables in Earth Engine from the [Import section](https://developers.google.com/earth-engine/importing).



## Random Samples

To get a collection of random points in a specified region, you can use:

In [55]:
# Define an arbitrary region in which to compute random points.
region = ee.Geometry.Rectangle(-119.224, 34.669, -99.536, 50.064)
center_coord = region.centroid().getInfo()['coordinates']
center_coord.reverse()

# Create 1000 random points in the region.
randomPoints = ee.FeatureCollection.randomPoints(region=region,points=100)

# Display the points.
Mapdisplay(center_coord,{'randomPoints':randomPoints.getInfo()},zoom_start=5)

# 7. FeatureCollection Information and Metadata

Methods for getting information from image collections are directly applicable to feature collections. (See the [ImageCollection Information and Metadata section](https://developers.google.com/earth-engine/ic_info) for details). You can use the aggregation shortcuts to count the number of features or summarize an attribute:



In [56]:
from pprint import pprint
# Load watersheds from a Fusion Table.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')

#Select just 10 features
sheds10 = sheds.limit(10)
sheds_centroid = sheds10.geometry().centroid().getInfo()['coordinates']
sheds_centroid.reverse()

# Display the table and print its first element.
pprint(sheds10.limit(1).getInfo()['features'][0]['properties'])

# Print the number of watersheds.
print('\n')
print('Count: ', sheds10.size().getInfo())

# Print stats for an area property.
print('\n')
pprint(sheds10.aggregate_stats('AreaSqKm').getInfo())

{'AreaAcres': 6633278.0,
 'AreaSqKm': 26843.95,
 'HUC6': 60300.0,
 'LoadDate': 1339401298000,
 'Shape_Area': 2.64484634184378,
 'Shape_Length': 11.5592015348736,
 'States': 'AL,TN',
 'TNMID': '{D529BEF5-4744-426F-A2B3-902503F81E26}',
 'description': '',
 'name': 'Middle Tennessee-Elk'}


Count:  10


{'type': 'DataDictionary',
 'values': {'max': 57877.79,
            'mean': 16816.704999999998,
            'min': 1285.22,
            'sample_sd': 17316.62981436133,
            'sample_var': 299865668.1276278,
            'sum': 168167.05,
            'sum_sq': 5526806683.7189,
            'total_count': 10,
            'total_sd': 16427.997483408166,
            'total_var': 269879101.314865,
            'valid_count': 10,
            'weight_sum': 10,
            'weighted_sum': 168167.05}}


In [61]:
Mapdisplay(sheds_centroid,{'sheds':sheds10.getInfo()},zoom_start=5)

For more general purpose FeatureCollection aggregation tools, see the Reducing a FeatureCollection section.



#8. Filtering a FeatureCollection

Filtering a `FeatureCollection` is analogous to filtering an `ImageCollection`. (See the Filtering an ImageCollection section). There are the `featureCollection.filterDate()`, and `featureCollection.filterBounds()` convenience methods and the `featureCollection.filter()` method for use with any applicable `ee.Filter`. For example:


In [0]:
# Load watersheds from a Fusion Table.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')

# Define a region roughly covering the continental US.
continentalUS = ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29)

# Filter the table geographically: only watersheds in the continental US.
filtered = sheds.filterBounds(continentalUS)

# Check the number of watersheds after filtering for location.
print('Count after filter:', filtered.size().getInfo())

# Filter to get only larger continental US watersheds.
largeSheds = filtered.filter(ee.Filter.gt('AreaSqKm', 25000))

# Check the number of watersheds after filtering for size and location.
print('Count after filtering by size:', largeSheds.size().getInfo())

Count after filter: 336
Count after filtering by size: 137


# 9. Mapping over a FeatureCollection

To apply the same operation to every `Feature` in a `FeatureCollection`, use `featureCollection.map()`. For example, to add another area attribute to every feature in a watersheds `FeatureCollection`, use:



In [0]:
# Load watersheds from a Fusion Table.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')

# This function computes the feature's geometry area and adds it as a property.
def addArea(feature):
  return feature.set({'areaHa': feature.geometry().area().divide(100 * 100)})

# Map the area getting function over the FeatureCollection.
areaAdded = sheds.map(addArea)

# Print the first feature from the collection with the added property.
print('First feature: \n')
pprint(areaAdded.first().getInfo())

In the previous example, note that a new property is set based on a computation with the feature’s geometry. Properties can also be set using a computation involving existing properties.

An entirely new `FeatureCollection` can be generated with `map()`. The following example converts the watersheds to centroids:

In [62]:
# This function creates a new feature from the centroid of the geometry.
def getCentroid(feature):
  # Keep this list of properties.
  keepProperties = ['name', 'HUC6', 'TNMID', 'AreaSqKm']
  # Get the centroid of the feature's geometry.
  centroid = feature.geometry().centroid()
  # Return a new Feature, copying properties from the old Feature.
  return ee.Feature(centroid).copyProperties(feature, keepProperties)


# Map the centroid getting function over the features.
centroids = sheds.map(getCentroid)
centroids_cn = centroids.geometry().centroid().getInfo()['coordinates']
centroids_cn.reverse()

# Display the results and print the first few.
Mapdisplay(centroids_cn,{'centroids':centroids.getInfo()},zoom_start=4)

In [0]:
print('Centroids: \n')
pprint(centroids.limit(5).getInfo())

Centroids: 

{'columns': {'AreaSqKm': 'Number',
             'HUC6': 'Number',
             'TNMID': 'String',
             'name': 'String'},
 'features': [{'geometry': {'coordinates': [-86.88398351334612,
                                            34.82011536641014],
                            'type': 'Point'},
               'id': '2',
               'properties': {'AreaSqKm': 26843.95,
                              'HUC6': 60300.0,
                              'TNMID': '{D529BEF5-4744-426F-A2B3-902503F81E26}',
                              'name': 'Middle Tennessee-Elk'},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-90.7083247171474,
                                            34.38423778799738],
                            'type': 'Point'},
               'id': '3',
               'properties': {'AreaSqKm': 1539.37,
                              'HUC6': 80201.0,
                              'TNMID': '{7DB15494-3B27-4A01-B0BC-03898F63BA95}',
 

Note that only a subset of properties is propagated to the features in the new collection.



# 10. Reducing a FeatureCollection

To aggregate data in the properties of a `FeatureCollection`, use `featureCollection.reduceColumns()`. For example, to check the area properties in the watersheds `FeatureCollection`, this code computes the Root Mean Square Error (RMSE) relative to the Earth Engine computed area:



In [0]:
# Load watersheds from a Fusion Table and filter to the continental US.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')\
          .filterBounds(ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29))

# This function computes the squared difference between an area property
# and area computed directly from the feature's geometry.
def areaDiff(feature):
  # Compute area in sq. km. directly from the geometry.
  area = feature.geometry().area().divide(1000 * 1000)
  # Compute the differece between computed area and the area property.
  diff = area.subtract(feature.get('AreaSqKm'))
  # Return the feature with the squared difference set to the 'diff' property.
  return feature.set('diff', diff.pow(2))


# Map the difference function over the collection.
rmse = ee.Number(sheds.map(areaDiff)\
                      .reduceColumns(ee.Reducer.mean(), ['diff'])\
                      .get('mean'))\
         .sqrt()
# Print the result.
print('RMSE=', rmse.getInfo())

RMSE= 4251.920829139865


In this example, note that the return value of reduceColumns() is a dictionary with key ‘mean’. To get the mean, cast the result of dictionary.get() to a number with ee.Number() before trying to call sqrt() on it. For more information about ancillary data structures in Earth Engine, see the [introduction of eeCourse]().

To overlay features on imagery, use featureCollection.reduceRegions(). For example, to compute the volume of precipitation in continental US watersheds, use reduceRegions() followed by a map():

In [0]:
# Load an image of daily precipitation in mm/day.
precip = ee.Image(ee.ImageCollection('NASA/ORNL/DAYMET_V3').first())

# Load watersheds from a Fusion Table and filter to the continental US.
sheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')\
          .filterBounds(ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29))

# Add the mean of each image as new properties of each feature.
withPrecip = precip.reduceRegions(sheds, ee.Reducer.mean())

# This function computes total rainfall in cubic meters.
def prcpVolume(feature):
  # Precipitation in mm/day -> meters -> sq. meters.
  volume = ee.Number(feature.get('prcp'))\
             .divide(1000)\
             .multiply(feature.geometry().area())
  return feature.set('volume', volume)

# 1. Map the function over the collection.
# 2. Sort descending.
# 3. Get only the 5 highest volume watersheds.
highVolume = withPrecip.map(prcpVolume)\
                       .sort('volume', False)\
                       .limit(5)

# Print the resulting FeatureCollection
pprint(highVolume.getInfo())  

For more information about reducing feature collections, see[ Statistics of FeatureCollection Columns]() and [Vector to Raster Conversion]().

# 11. Vector to Raster Interpolation

Interpolation from vector to raster in Earth Engine creates an `Image` from a `FeatureCollection`. Specifically, Earth Engine uses numeric data stored in a property of the features to interpolate values at new locations outside of the features. The interpolation results in a continuous `Image` of interpolated values up to the distance specified.



## Inverse Distance Weighted Interpolation

The inverse distance weighting (IDW) function in Earth Engine is based on the method described by [Basso et al. (1999)](http://ieeexplore.ieee.org/xpls/abs_all.jsp?arnumber=805606&tag=1). An additional control parameter is added in the form of a decay factor (gamma) on the inverse distance. Other parameters include the mean and standard deviation of the property to interpolate and the maximum range distance over which to interpolate. The following example creates an interpolated surface of [PM 2.5](https://www.epa.gov/pm-pollution/particulate-matter-pm-basics) concentrations from a `FeatureCollection` of points representing PM 2.5 measurement stations.

In [63]:
# Load a Fusion Table corresponding to mean annual PM2.5 concentrations at points.
airQualityMeasurements = ee.FeatureCollection('ft:14BLob4jGA6au2MB1cx0GhxYDvZc-lVLAWhqBAZuN')
airQualityMeasurements_center = airQualityMeasurements.geometry().centroid().getInfo()['coordinates']
airQualityMeasurements_center.reverse()

Mapdisplay(airQualityMeasurements_center,{'airQuality':airQualityMeasurements.getInfo()},zoom_start=4)

In [65]:
# This is the name of the property to interpolate.
propertyToInterpolate = 'ArithmeticMean'

# Combine mean and SD reducers for efficiency.
combinedReducer = ee.Reducer.mean()\
                            .combine(reducer2=ee.Reducer.stdDev(),
                                     sharedInputs=True)

# Estimate global mean and standard deviation (SD) from the points.
stats = airQualityMeasurements.reduceColumns(reducer=combinedReducer,
                                             selectors= [propertyToInterpolate])

# Do the interpolation, valid to 50 kilometers.
interpolatedPM25 = airQualityMeasurements.inverseDistance(range=50 * 1000,
                                                          propertyName=propertyToInterpolate,
                                                          mean=stats.get('mean'),
                                                          stdDev=stats.get('stdDev'),
                                                          gamma=0.5)

# Visualize the resulting interpolated raster.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
vis = {'min': 0, 'max': 15, 'palette': ['blue', 'green', 'red']}
mapid = interpolatedPM25.getMapId(vis)

Mapdisplay(center=airQualityMeasurements_center,
           dicc={'PM2.5 - IDW':mapid},
           zoom_start=4)

Note that, as specified by the range parameter, the interpolation only exists up to 50 kilometers from the nearest measurement station.

##Kriging

[Kriging](https://en.wikipedia.org/wiki/Kriging) is an interpolation method that uses a modeled estimate of [semi-variance](https://en.wikipedia.org/wiki/Semivariance) to create an image of interpolated values that is an optimal combination of the values at known locations. The Kriging estimator requires parameters that describe the shape of a [semi-variogram](https://en.wikipedia.org/wiki/Variogram) fit to the known data points. These parameters are illustrated by Figure 1.

<center>
<img src= "https://developers.google.com/earth-engine/images/Variogram.png" width=60%>
</center>

Figure 1. The nugget, sill and range parameters illustrated on a idealized variogram function.

The following example samples a sea surface temperature (SST) image at random locations, then interpolates SST from the sample using Kriging:

In [66]:
# Load an image of sea surface temperature (SST).
sst = ee.Image('NOAA/AVHRR_Pathfinder_V52_L3/20120802025048')\
        .select('sea_surface_temperature')\
        .rename('sst')\
        .divide(100)

# Define a geometry in which to sample points
geometry = ee.Geometry.Rectangle([-65.60, 31.75, -52.18, 43.12])

# Sample the SST image at 1000 random locations.

def get_Feature(sample):
  lat = sample.get('latitude')
  lon = sample.get('longitude')
  sst = sample.get('sst')
  return ee.Feature(ee.Geometry.Point([lon, lat]), {'sst': sst})

samples = sst.addBands(ee.Image.pixelLonLat())\
             .sample(region=geometry,numPixels= 1000)\
             .map(get_Feature)


# Interpolate SST from the sampled points.
interpolated = samples.kriging(propertyName='sst',
                               shape='exponential',
                               range=100 * 1000,
                               sill=1.0,
                               nugget=0.1,
                               maxDistance=100 * 1000,
                               reducer='mean')

# Visualize the resulting interpolated raster.
colors = ['00007F', '0000FF', '0074FF',
          '0DFFEA', '8CFF41', 'FFDD00',
          'FF3700', 'C30000', '790000']
vis = {'min':-3, 'max':40, 'palette': colors}
mapid = interpolated.getMapId(vis)

Mapdisplay(center=airQualityMeasurements_center,
           dicc={'PM2.5 - KRIGING':mapid},
           zoom_start=4)

The size of the neighborhood in which to perform the interpolation is specified by the maxDistance parameter. Larger sizes will result in smoother output but slower computations.

<!--NAVIGATION-->
 < [ImageCollection](3_eeImageCollection.ipynb) | [Contents](index.ipynb) |  [Reducer](5_Reducer.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/4_features.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>